# Lasso Regression w/ Coordinate Descent from scratch using Numpy

Objective: To emulate sklearn.linear_model.Lasso() using numpy

The data that will be used in this notebook will be the mystery.dat file from the UCSanDiego ML course on edX. Use the [link here](https://courses.edx.org/courses/course-v1:UCSanDiegoX+DSE220x+1T2019/courseware/bcbbed9cb9dc4157ba9e7e608dddb0ac/0290999d30f74caab3c9e5fb955d29fc/1?activate_block_id=block-v1%3AUCSanDiegoX%2BDSE220x%2B1T2019%2Btype%40vertical%2Bblock%4015741288db67451991e67be793b369d0) to navigate to the file and download it.

Before you start running codes on this notebook, place the mystery.dat file in the same directory.

## 1. Set up notebook and load data set

In [1]:
import numpy as np

data = np.genfromtxt('mystery.dat', delimiter = ',')



## 2. Refresher on Lasso Regression

The lasso regression loss function is from the edX course is defined as

\begin{equation*}
L(w,b) = \left(\sum_{i=1}^n (y^{(i)} - (w \cdot x^{(i)} + b))^2\right) + λ||w||_1
\end{equation*}

$$||w||_1  = |w_1| + |w_2| + |w_3| + ... $$

**The goal is to minimise this loss function by adjusting the weights i.e. the vector w.**

From scikit-learn's description in Section 1.1.3 [here](https://scikit-learn.org/stable/modules/linear_model.html), the lasso regression loss function is defined as 

$$ \frac{1}{2N}||y - Xw||^2_2 + α||w||_1 $$

The equation above can be re-written as follows to aid understanding

$$ L(w) = \frac{1}{2N}\left(\sum_{i=1}^n (y^{(i)} - \tilde{w} \cdot \tilde{x}^{(i)})^2\right) + α\left(\sum_{j=1}^d |w_j|\right)$$

$$\tilde{w} = [w_0, w_1, w_2, ..., w_d]$$
$$\tilde{x} = [1, x_1, x_2, ..., x_d]$$

Note that $w_{0}$ will be the intercept which is b in the loss function from the edX course. Besides that, α is the equivalent to λ.

The difference between the edX's loss function and scikit's is the appearance of 1/2N in front of the first summation term. N represents the number of observations/samples i.e. number of rows in the dataset.

**This loss function is optimised using coordinate descent**

## 3. Deriving the coordinate descent used in lasso regression

Coordinate descent is an optimisation algorithm which goal is to minimise some function. The diagram below best describes what coordinate descent does:

![Coordinate Descent Illustration](coordinate_descent.png)
Source: Machine Learning Course from University of Washington on Coursera

From the illustration above, the contours represent the value of the loss function where the value at the center is the lowest (imagine a convex function). Coordinate descent works by choosing one feature to optimise while keeping the other features the same.

**Coordinate descent**:

Initialise w = 0
>while not converged 
>>pick a coordinate j, update j by minimising the loss function by altering the value of j

When assessing the convergence, it is up to the user to set how small of the step size it wants the coordinate descent to take to be considered as a convergence.

Note from the illustration above that the coordinate descent takes smaller and smaller steps to reach the minimum value of the cost function.

We can measure the maximum step size taken during each iteration of the descent, and when the maximum step size is lower than a tolerance level (set by the user), we say that the coordinate descent has converged. 

#### Deriving the partial derivative of the MSE term in the loss function


Ignoring the regulariser term α, let the focus be on the mean square error (MSE) of the loss function

$$ L(w) = \frac{1}{2N}\left(\sum_{i=1}^n (y^{(i)} - \tilde{w} \cdot \tilde{x}^{(i)})^2\right) $$

Partially differentiating the above equation with $w_j$ yields:

$$ \frac{𝜕L(w)}{𝜕w_j} = \frac{1}{2N}\left(\sum_{i=1}^n 2(y^{(i)} - \tilde{w} \cdot \tilde{x}^{(i)})(-x_j)\right) $$

To apply coordinate descent, the $\tilde{w} \cdot \tilde{x}^{(i)}$ is rewritten: 

$$ \tilde{w} \cdot \tilde{x}^{(i)} = \left(\sum_{k≠j}^n \tilde{w}_k \cdot \tilde{x}^{(i)}_k\right) + \tilde{w}_j\tilde{x}^{(i)}_j $$

Expanding/Summarising the partial differential equation:

$$ \frac{𝜕L(w)}{𝜕w_j} = \frac{1}{N}\left(\sum_{i=1}^n (-\tilde{x}^{(i)}_j)(y^{(i)} - \left(\sum_{k≠j}^n \tilde{w}_k \cdot \tilde{x}^{(i)}_k\right)) \right) + \frac{1}{N}\tilde{w}_j\left(\sum_{i=1}^n \tilde{x}^{(i)}_j \right)^2    $$

Let us define the terms $ρ_j$ and $z_j$:

$$ ρ_j = \left(\sum_{i=1}^n (\tilde{x}^{(i)}_j)(y^{(i)} - \left(\sum_{k≠j}^n \tilde{w}_k \cdot \tilde{x}^{(i)}_k\right)) \right)$$

$$ z_j = \left(\sum_{i=1}^n \tilde{x}^{(i)}_j \right)^2$$



Therefore, the final partial derivative equation is: 

$$ \frac{𝜕L(w)}{𝜕w_j} = \frac{-1}{N}ρ_j + \frac{1}{N}\tilde{w}_jz_j$$

To minimise the loss function, the partial derivative of the loss function is set to 0.

Hence, the optimum solution for the MSE part of the loss functions is:

$$ w_j = \frac{ρ_j}{z_j}$$

#### Deriving the partial derivative of the regulariser term in the loss function

To partially differentiate the regularise term in the loss function, let's first look at the graph of $|w_j|$

![w_j modulus](w_j_modulus.png)

No derivative exists at $|w_j| = 0$!! Or rather, there is no gradient for the sharp point of that graph. 

From here, we have to look into sub-gradients of $|w_j|$:
![subgradient](subgradient.png)

The sub-gradients of $|w_j|$ are annotated by V which is between -1 and 1.

To better understand subgradients, refer to [this video at time 4:11](https://www.coursera.org/lecture/ml-regression/deriving-the-lasso-coordinate-descent-update-6OLyn)

Hence, the partial derivations of the regulariser term in the loss function are :

$$ 
α𝜕_{w_j}\left(\sum_{j=1}^d |w_j|\right) = α\frac{𝜕|w_j|}{𝜕w_j} = 
    \begin{cases}
        -α & \text{when $w_j$ < 0}\\
        [-α,α] & \text{when $w_j$ = 0}\\
        α & \text{when $w_j$ > 0}\\
    \end{cases}
$$


#### Solving the partial derivation of the total Lasso regression loss function

The partial derivative of the rewritten loss function in section 2 is:

$$
\frac{𝜕L(w)}{𝜕w_j} = \frac{-1}{N}ρ_j + \frac{1}{N}\tilde{w}_jz_j + 
    \begin{cases}
        -α & \text{when $w_j$ < 0}\\
        [-α,α] & \text{when $w_j$ = 0}\\
        α & \text{when $w_j$ > 0}\\
    \end{cases}
$$

When setting the partial derivative to 0, there are 3 cases to consider:

**Case 1 :** $w_j$ < 0
$$
\frac{-1}{N}ρ_j + \frac{1}{N}\tilde{w}_jz_j - α = 0
$$

$$
w_j = \frac{ρ_j + Nα}{z_j}
$$

For $w_j$ < 0
$$
ρ_j < -Nα
$$

**Case 2 :** $w_j$ = 0
$$
\frac{-1}{N}ρ_j + \frac{1}{N}\tilde{w}_jz_j + [-α,α] = 0
$$

$$
w_j = [\frac{ρ_j - Nα}{z_j},\frac{ρ_j + Nα}{z_j}]
$$

For $w_j$ = 0, need $[\frac{ρ_j - Nα}{z_j},\frac{ρ_j + Nα}{z_j}]$ to contain 0, hence
$$
-Nα ≤ ρ_j ≤ Nα 
$$

**Case 3 :** $w_j$ > 0
$$
\frac{-1}{N}ρ_j + \frac{1}{N}\tilde{w}_jz_j + α = 0
$$

$$
w_j = \frac{ρ_j - Nα}{z_j}
$$

For $w_j$ > 0
$$
ρ_j > Nα
$$

Note that $w_0$ is not part of the regulariser term, hence it is updated using the MSE partial derivative term, therefore:

$$ w_0 = \frac{ρ_0}{z_0}$$

## 4. The coordinate descent algorithm used here

1. Pre-compute:
$$ z_j = \left(\sum_{i=1}^n \tilde{x}^{(i)}_j \right)^2$$

2. Initialise the $\tilde{w}$ to be a zero vector

3. Initialise the maximum step parameter and tolerance level
> Can just set the maximum step to any large value at the beginning

4. While the algorithm has not converged
> For j = 0,1,...,D
>> Compute:
$$ ρ_j = \left(\sum_{i=1}^n (\tilde{x}^{(i)}_j)(y^{(i)} - \left(\sum_{k≠j}^n \tilde{w}_k \cdot \tilde{x}^{(i)}_k\right)) \right)$$
>> If j = 0
$$ w_j = \frac{ρ_j}{z_j}$$
>> Else
$$ w_j = 
    \begin{cases}
        \frac{ρ_j + Nα}{z_j} & \text{if $ρ_j$ < -Nα}\\
        0 & \text{if -Nα ≤ $ρ_j$ ≤ Nα }\\
        \frac{ρ_j - Nα}{z_j} & \text{if $ρ_j$ > Nα}\\
    \end{cases}
$$

## 5. numpy coding to perform LASSO regression on mystery.dat

In [2]:
#Create a function to get the output
def predicted_values(X, w):
    # X will be n x (d+1)
    # w will be (d+1) x 1
    predictions = np.matmul(X,w) # n x 1
    return(predictions)

In [3]:
#rho computation
def rho_compute(y,X,w,j):
    #y is the response variable
    #X is the predictor variables matrix
    #w is the weight vector
    #j is the feature selector
    X_k = np.delete(X,j,1) #Remove the j variable i.e. j column
    w_k = np.delete(w,j) #Remove the weight j
    predict_k = predicted_values(X_k, w_k)
    residual = y - predict_k
    rho_j = np.sum(X[:,j]*residual)
    return(rho_j)

In [4]:
#z computation for unnormalised features
def z_compute(X):
    z_vector = np.sum(X*X, axis = 0) #Compute sum for each column
    return(z_vector)

In [5]:
def coordinate_descent(y,X,w,alpha,z,tolerance):
    max_step = 100.
    iteration = 0
    while(max_step > tolerance):
        iteration += 1
        #print("Iteration (start) : ",iteration)
        old_weights = np.copy(w)
        #print("\nOld Weights\n",old_weights)
        for j in range(len(w)): #Take the number of features ie columns
            rho_j = rho_compute(y,X,w,j)
            if j == 0: #Intercept is not included with the alpha regularisation
                w[j] = rho_j/z[j]
            elif rho_j < -alpha*len(y):
                w[j] = (rho_j + (alpha*len(y)))/z[j]
            elif rho_j > -alpha*len(y) and rho_j < alpha*len(y):
                w[j] = 0.
            elif rho_j > alpha*len(y):
                w[j] = (rho_j - (alpha*len(y)))/z[j]
            else:
                w[j] = np.NaN
        #print("\nNew Weights\n",w)
        step_sizes = abs(old_weights - w)
        #print("\nStep sizes\n",step_sizes)
        max_step = step_sizes.max()
        #print("\nMax step:",max_step)
        
        
    return(w, iteration, max_step)

In [6]:
#Initialise the data

#101 rows for both input (x) and output (y)
x = data[:,0:100] # 100 predictors - columns 0 to 99
y = data[:,100] # 1 response variable - column 100

#Obtain feature matrix by adding column of 1s to input matrix x
X = np.column_stack((np.ones((x.shape[0],1)),x)) #101 columns

#Initialise weight/parameter vector, w, to be a zero vector
w = np.zeros(X.shape[1], dtype = float)

#Pre-compute the z_j term
z = z_compute(X)

#Set the alpha and tolerance level
alpha = 0.1
tolerance = 0.0001

#Obtain the following from the coordinate descent:
#1. Optimum weight parameter
#2. Number of iterations
#3. Maximum step size at the last iteration

w_opt, iterations, max_step = coordinate_descent(y,X,w,alpha,z,tolerance)

In [7]:
#Print out the optimised weights
np.set_printoptions(precision = 3, suppress = True)
print("Intercept is:",w_opt[0])
print("\nCoefficients are:\n",w_opt[1:101])
print("\nNumber of iterations is:",iterations)

Intercept is: -0.12284593210597963

Coefficients are:
 [ 0.     0.836  0.666  0.     1.009 -0.031  0.888 -0.229  0.     0.
  0.887  0.     0.702  0.    -0.116 -0.079  0.86   0.     0.779  0.039
  0.04   0.     0.841  0.    -0.013  0.     0.     0.     0.547  0.
  0.     0.     0.     0.     0.     0.    -0.037  0.     0.     0.
  0.     0.     0.    -0.065  0.     0.     0.    -0.004  0.047  0.017
  0.     0.     0.     0.013 -0.033  0.086  0.062 -0.044  0.     0.
  0.     0.     0.     0.     0.     0.     0.    -0.042  0.     0.097
  0.     0.     0.017  0.     0.     0.011  0.     0.075  0.     0.
  0.15   0.     0.     0.     0.     0.     0.     0.     0.    -0.068
  0.     0.078  0.     0.     0.     0.     0.     0.    -0.059  0.   ]

Number of iterations is: 17


In [8]:
#Sort to see which are the most important features
values = np.sort(abs(w_opt[1:101]))[::-1] 
index = np.argsort(abs(w_opt[1:101]))[::-1] + 1 #Add 1 to not show zero-index

np.set_printoptions(precision = 3, suppress = True)
print(np.column_stack((index,values)))

[[  5.      1.009]
 [  7.      0.888]
 [ 11.      0.887]
 [ 17.      0.86 ]
 [ 23.      0.841]
 [  2.      0.836]
 [ 19.      0.779]
 [ 13.      0.702]
 [  3.      0.666]
 [ 29.      0.547]
 [  8.      0.229]
 [ 81.      0.15 ]
 [ 15.      0.116]
 [ 70.      0.097]
 [ 56.      0.086]
 [ 16.      0.079]
 [ 92.      0.078]
 [ 78.      0.075]
 [ 90.      0.068]
 [ 44.      0.065]
 [ 57.      0.062]
 [ 99.      0.059]
 [ 49.      0.047]
 [ 58.      0.044]
 [ 68.      0.042]
 [ 21.      0.04 ]
 [ 20.      0.039]
 [ 37.      0.037]
 [ 55.      0.033]
 [  6.      0.031]
 [ 50.      0.017]
 [ 73.      0.017]
 [ 25.      0.013]
 [ 54.      0.013]
 [ 76.      0.011]
 [ 48.      0.004]
 [ 24.      0.   ]
 [  4.      0.   ]
 [ 39.      0.   ]
 [ 38.      0.   ]
 [  9.      0.   ]
 [ 10.      0.   ]
 [ 12.      0.   ]
 [ 36.      0.   ]
 [ 35.      0.   ]
 [ 34.      0.   ]
 [ 18.      0.   ]
 [ 14.      0.   ]
 [ 41.      0.   ]
 [ 33.      0.   ]
 [ 32.      0.   ]
 [ 31.      0.   ]
 [ 30.      

In [9]:
#Compare with sklearn's Lasso
from sklearn import linear_model
mystery = linear_model.Lasso(alpha = 0.1, tol = 0.0001) #alpha is just the regulariser term
mystery.fit(x,y)

print("sklearn Lasso intercept :",mystery.intercept_)
print("\nsklearn Lasso coefficients :\n",mystery.coef_)
print("\nsklearn Lasso number of iterations :",mystery.n_iter_)

sklearn Lasso intercept : -0.12281129097551671

sklearn Lasso coefficients :
 [ 0.     0.836  0.666  0.     1.009 -0.031  0.888 -0.229 -0.    -0.
  0.887 -0.     0.702 -0.    -0.116 -0.079  0.86  -0.     0.779  0.039
  0.04   0.     0.841 -0.    -0.013 -0.     0.    -0.     0.547  0.
 -0.     0.     0.    -0.    -0.    -0.    -0.037  0.    -0.     0.
 -0.     0.    -0.    -0.065  0.     0.    -0.    -0.004  0.047  0.017
 -0.     0.     0.     0.013 -0.033  0.086  0.062 -0.044  0.     0.
  0.     0.     0.     0.    -0.     0.     0.    -0.042  0.     0.097
  0.     0.     0.017  0.    -0.     0.011 -0.     0.075  0.    -0.
  0.15   0.    -0.     0.    -0.    -0.    -0.     0.    -0.    -0.068
  0.     0.078  0.    -0.    -0.     0.     0.    -0.    -0.059  0.   ]

sklearn Lasso number of iterations : 16


In [10]:
values = np.sort(abs(mystery.coef_))[::-1]
index = np.argsort(abs(mystery.coef_))[::-1] + 1

np.set_printoptions(precision = 3, suppress = True)
print(np.column_stack((index,values)))

[[  5.      1.009]
 [  7.      0.888]
 [ 11.      0.887]
 [ 17.      0.86 ]
 [ 23.      0.841]
 [  2.      0.836]
 [ 19.      0.779]
 [ 13.      0.702]
 [  3.      0.666]
 [ 29.      0.547]
 [  8.      0.229]
 [ 81.      0.15 ]
 [ 15.      0.116]
 [ 70.      0.097]
 [ 56.      0.086]
 [ 16.      0.079]
 [ 92.      0.078]
 [ 78.      0.075]
 [ 90.      0.068]
 [ 44.      0.065]
 [ 57.      0.062]
 [ 99.      0.059]
 [ 49.      0.047]
 [ 58.      0.044]
 [ 68.      0.042]
 [ 21.      0.04 ]
 [ 20.      0.039]
 [ 37.      0.037]
 [ 55.      0.033]
 [  6.      0.031]
 [ 50.      0.017]
 [ 73.      0.017]
 [ 25.      0.013]
 [ 54.      0.013]
 [ 76.      0.011]
 [ 48.      0.004]
 [ 24.      0.   ]
 [  4.      0.   ]
 [ 39.      0.   ]
 [ 38.      0.   ]
 [  9.      0.   ]
 [ 10.      0.   ]
 [ 12.      0.   ]
 [ 36.      0.   ]
 [ 35.      0.   ]
 [ 34.      0.   ]
 [ 18.      0.   ]
 [ 14.      0.   ]
 [ 41.      0.   ]
 [ 33.      0.   ]
 [ 32.      0.   ]
 [ 31.      0.   ]
 [ 30.      

In [11]:
#Comparison within the same cell

print("sklearn Lasso intercept :",mystery.intercept_)
print("\nsklearn Lasso coefficients :\n",mystery.coef_)
print("\nsklearn Lasso number of iterations :",mystery.n_iter_)
print("\n------------------------------------------------------------------------\n")
print("Intercept is:",w_opt[0])
print("\nCoefficients are:\n",w_opt[1:101])
print("\nNumber of iterations is:",iterations)

sklearn Lasso intercept : -0.12281129097551671

sklearn Lasso coefficients :
 [ 0.     0.836  0.666  0.     1.009 -0.031  0.888 -0.229 -0.    -0.
  0.887 -0.     0.702 -0.    -0.116 -0.079  0.86  -0.     0.779  0.039
  0.04   0.     0.841 -0.    -0.013 -0.     0.    -0.     0.547  0.
 -0.     0.     0.    -0.    -0.    -0.    -0.037  0.    -0.     0.
 -0.     0.    -0.    -0.065  0.     0.    -0.    -0.004  0.047  0.017
 -0.     0.     0.     0.013 -0.033  0.086  0.062 -0.044  0.     0.
  0.     0.     0.     0.    -0.     0.     0.    -0.042  0.     0.097
  0.     0.     0.017  0.    -0.     0.011 -0.     0.075  0.    -0.
  0.15   0.    -0.     0.    -0.    -0.    -0.     0.    -0.    -0.068
  0.     0.078  0.    -0.    -0.     0.     0.    -0.    -0.059  0.   ]

sklearn Lasso number of iterations : 16

------------------------------------------------------------------------

Intercept is: -0.12284593210597963

Coefficients are:
 [ 0.     0.836  0.666  0.     1.009 -0.031  0.888 -0.22

In [12]:
w_opt[1:101]

array([ 0.   ,  0.836,  0.666,  0.   ,  1.009, -0.031,  0.888, -0.229,
        0.   ,  0.   ,  0.887,  0.   ,  0.702,  0.   , -0.116, -0.079,
        0.86 ,  0.   ,  0.779,  0.039,  0.04 ,  0.   ,  0.841,  0.   ,
       -0.013,  0.   ,  0.   ,  0.   ,  0.547,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   , -0.037,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   , -0.065,  0.   ,  0.   ,  0.   , -0.004,
        0.047,  0.017,  0.   ,  0.   ,  0.   ,  0.013, -0.033,  0.086,
        0.062, -0.044,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   , -0.042,  0.   ,  0.097,  0.   ,  0.   ,
        0.017,  0.   ,  0.   ,  0.011,  0.   ,  0.075,  0.   ,  0.   ,
        0.15 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   , -0.068,  0.   ,  0.078,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   , -0.059,  0.   ])

In [13]:
print("Values are compared to sklearn's Lasso results:")
np.set_printoptions(precision = 10, suppress = False)
error = np.zeros(len(w_opt[1:101]), dtype = float)
for j in range(len(w_opt[1:101])):
    if(w_opt[1:101][j] == 0 and mystery.coef_[j] == 0):
        error[j] = 0.
    else:
        error[j] = abs(((w_opt[1:101][j] - mystery.coef_[j])/mystery.coef_[j])*100)

print("Maximum difference in coefficients (%):",np.max(error))
print("Average difference in coefficients (%):",np.mean(error))

intercept_diff = abs(((w_opt[0] - mystery.intercept_)/mystery.intercept_)*100)
print("Difference in intercept (%):",intercept_diff)



Values are compared to sklearn's Lasso results:
Maximum difference in coefficients (%): 0.11279634972984848
Average difference in coefficients (%): 0.007858722135127718
Difference in intercept (%): 0.028206796124165913


## 6. Conclusion

Attempt to emulate sklearn.linear_model.Lasso() function using numpy only is rather successful. Using sklearn.linear_model.Lasso(), the coding in numpy managed to replicate with results with very minimal error. 
